# Notebook 4: Inference and App Preparation

### Objectives:
1.  **Load the Best Model:** Load the saved weights of our winning model (`ResNetUNet`).
2.  **Create an Inference Pipeline:** Develop a robust function that takes a single image file, preprocesses it, and runs it through the model to get a segmentation mask.
3.  **Test the Pipeline:** Verify that our inference function works correctly on a sample image, creating the final logic we'll use in our Streamlit application.

## 1. Setup, Imports, and Path Definitions

First, we import the necessary libraries and define our project paths. We also import our model architectures from the `src` directory.

In [ ]:
import os
import sys
import numpy as np
import torch
import torch.nn as nn
from PIL import Image
import matplotlib.pyplot as plt
import shutil  # Added for copying files

# --- Define Project Directories ---
try:
    ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
except NameError:
    ROOT_DIR = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))

# --- Add Project Root to Python Path ---
if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)

print(f"Project Root added to path: {ROOT_DIR}")

# --- Import our custom models ---
from src.models import BaselineUNet, ResNetUNet, TransUNet

MODELS_DIR = os.path.join(ROOT_DIR, 'models')
FIGURES_DIR = os.path.join(ROOT_DIR, 'figures')

# --- Set Device ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Root Directory: {ROOT_DIR}")
print(f"Models Directory: {MODELS_DIR}")
print(f"Device: {DEVICE}")

## 2. Load the Best Trained Model

Based on our benchmark results, the `ResNetUNet` was the clear winner. We will now load its saved weights into a model instance and set it to evaluation mode.

In [ ]:
BEST_MODEL_NAME = 'ResNetUNet'
model_path = os.path.join(MODELS_DIR, f'{BEST_MODEL_NAME}_best_model.pth')

# Instantiate the model architecture
model = ResNetUNet(in_channels=4, out_channels=1).to(DEVICE)

# Load the saved weights
model.load_state_dict(torch.load(model_path, map_location=torch.device(DEVICE)))

# Set the model to evaluation mode
model.eval()

print(f"Successfully loaded best model: {BEST_MODEL_NAME} from {model_path}")

## 3. Create the Inference Pipeline

Here we create a single, powerful function called `predict`. This function encapsulates all the necessary steps: opening an image, preprocessing it to match the training format, running inference, and post-processing the output mask. This is the core function we will use in our Streamlit app.

In [ ]:
# =========================================================================
#             CREATE THE INFERENCE PIPELINE (CORRECTED)
# =========================================================================

def predict(model, npy_image_path, device):
    """
    Runs the full inference pipeline on a single, preprocessed .npy image slice.
    """
    # Load the correctly preprocessed numpy array
    img_np = np.load(npy_image_path)
    
    # Convert to PyTorch tensor and add batch dimension
    input_tensor = torch.from_numpy(img_np).permute(2, 0, 1).unsqueeze(0).to(device)
    
    model.eval()
    with torch.no_grad():
        # Get raw prediction (logits)
        logits = model(input_tensor)
        # Convert to probabilities -> binary mask
        pred_mask = (torch.sigmoid(logits) > 0.5).float()
        
    # Remove batch dimension and move to CPU
    pred_mask = pred_mask.squeeze(0).cpu().numpy()
    
    return pred_mask


## 4. Creating Sample Images for App Deployment & Testing the Pipeline
This cell takes a few images from our test set and saves them as user-friendly .png files for the application demo.

In [ ]:
# =========================================================================
#       FINAL CELL: CREATE SAMPLES & TEST INFERENCE PIPELINE (DEPLOYMENT READY)
# This version COPIES the .npy files to streamlit_app/sample_images for deployment
# =========================================================================

from sklearn.model_selection import train_test_split
from glob import glob
import random
import json

print("--- Creating Sample Images & Copying Data for Deployment ---")

# --- 1. Re-create the test set file list ---
PROCESSED_DATA_DIR = os.path.join(ROOT_DIR, 'data', 'processed')
all_files = glob(os.path.join(PROCESSED_DATA_DIR, "*_image.npy"))
_, test_files = train_test_split(all_files, test_size=0.15, random_state=42)

# --- 2. Define the sample directory ---
SAMPLE_IMAGES_DIR = os.path.join(ROOT_DIR, 'streamlit_app', 'sample_images')
os.makedirs(SAMPLE_IMAGES_DIR, exist_ok=True)
print(f"Samples will be saved to: {SAMPLE_IMAGES_DIR}")

# --- 3. Select random test files ---
num_samples_to_create = 5
random_test_npy_paths = random.sample(test_files, num_samples_to_create)

# --- 4. Loop, Copy Files, Create PNGs, Predict, and Visualize ---
sample_mapping = {}

for i, original_npy_path in enumerate(random_test_npy_paths):
    
    # --- a. Copy the .npy files to streamlit_app/sample_images ---
    # Copy image file
    image_filename = f"sample_{i+1}_image.npy"
    mask_filename = f"sample_{i+1}_mask.npy"
    
    copied_image_path = os.path.join(SAMPLE_IMAGES_DIR, image_filename)
    copied_mask_path = os.path.join(SAMPLE_IMAGES_DIR, mask_filename)
    
    # Copy the actual .npy files
    shutil.copy2(original_npy_path, copied_image_path)
    shutil.copy2(original_npy_path.replace("_image.npy", "_mask.npy"), copied_mask_path)
    
    print(f"Copied: {os.path.basename(original_npy_path)} -> {image_filename}")
    
    # --- b. Run INFERENCE on the COPIED .npy file ---
    predicted_mask_array = predict(model, copied_image_path, DEVICE)
    
    # --- c. Create the .png files for visualization ---
    mri_slice = np.load(copied_image_path)[:, :, 0]  # T1c channel
    mask_slice = (np.load(copied_mask_path) > 0).astype(np.uint8)

    img_to_save = Image.fromarray((mri_slice * 255).astype(np.uint8))
    mask_to_save = Image.fromarray(mask_slice * 255)

    img_save_path = os.path.join(SAMPLE_IMAGES_DIR, f"sample_{i+1}.png")
    mask_save_path = os.path.join(SAMPLE_IMAGES_DIR, f"sample_{i+1}_mask.png")
    img_to_save.save(img_save_path)
    mask_to_save.save(mask_save_path)
    
    # --- d. Update mapping to point to copied files (deployment-ready paths) ---
    sample_mapping[f"sample_{i+1}.png"] = {
        "image_npy": image_filename,  # Just filename, not full path
        "mask_npy": mask_filename,
        "original_source": os.path.basename(original_npy_path)
    }
    
    # --- e. Visualize the result ---
    print(f"\n--- Verification for sample_{i+1}.png ---")
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
    
    ax1.imshow(img_to_save, cmap='bone'); ax1.set_title('Input Image'); ax1.axis('off')
    ax2.imshow(mask_to_save, cmap='magma'); ax2.set_title('Ground Truth Mask'); ax2.axis('off')
    ax3.imshow(predicted_mask_array.squeeze(), cmap='magma'); ax3.set_title(f'Predicted Mask'); ax3.axis('off')
    
    plt.show()

# --- 5. Save the deployment-ready mapping ---
mapping_path = os.path.join(SAMPLE_IMAGES_DIR, 'sample_mapping.json')
with open(mapping_path, 'w') as f:
    json.dump(sample_mapping, f, indent=2)

print(f"\n--- Sample mapping saved to: {mapping_path} ---")
print("Deployment-ready mapping contents:")
for png, info in sample_mapping.items():
    print(f"  {png} -> {info['image_npy']} (from {info['original_source']})")

# --- 6. Show what files are now in the sample_images directory ---
print(f"\n--- Files in {SAMPLE_IMAGES_DIR} (ready for deployment): ---")
for file in sorted(os.listdir(SAMPLE_IMAGES_DIR)):
    file_path = os.path.join(SAMPLE_IMAGES_DIR, file)
    size_mb = os.path.getsize(file_path) / (1024 * 1024)
    print(f"  {file} ({size_mb:.2f} MB)")

print(f"\n--- {num_samples_to_create} samples created and copied for deployment! ---")
print("✅ All .npy files are now in streamlit_app/sample_images and can be deployed")

## End of Notebook 4

This concludes our inference preparation. We have successfully:
- Loaded our best-performing `ResNetUNet` model.
- Built and tested a robust `predict` function that handles the full preprocessing and inference pipeline.

We are now fully prepared to build the interactive Streamlit application.